# Grid Search for Ternary Classification

This notebook performs grid search optimization for the ternary classification model.
It searches over:

1. **Labeling parameters**: horizon, gauss_window, gauss_std
2. **Feature parameters**: orb_multiplier, coord_mode (geo/helio/both)
3. **Model hyperparameters**: max_depth, learning_rate, weight_power

Uses caching to avoid recomputation of intermediate data.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# ENVIRONMENT CHECK
# ═══════════════════════════════════════════════════════════════════════════════════
import sys
print(f"Python: {sys.version}")

## 1. Setup & Configuration

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# IMPORTS
# ═══════════════════════════════════════════════════════════════════════════════════
import itertools
import time
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Any, Optional

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# RESEARCH modules
from RESEARCH.config import cfg, PROJECT_ROOT
from RESEARCH.data_loader import load_market_data
from RESEARCH.labeling import create_ternary_labels
from RESEARCH.astro_engine import (
    init_ephemeris,
    calculate_bodies_for_dates_multi,
    calculate_aspects_for_dates,
    calculate_phases_for_dates,
)
from RESEARCH.features import (
    build_full_features,
    merge_features_with_labels,
    get_feature_columns,
)
from RESEARCH.model_training import (
    split_dataset,
    prepare_xy,
    train_xgb_model,
    calc_metrics,
    check_cuda_available,
    compute_stronger_weights,
)
from RESEARCH.cache_utils import (
    save_cache,
    load_cache,
    cache_exists,
    list_cache,
    get_cache_path,
)

print("✓ All modules imported")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# GRID SEARCH CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════════════
# Define all parameter grids to search over.
# Keep grids small for initial exploration, expand later.
# ═══════════════════════════════════════════════════════════════════════════════════

# ─────────────────────────────────────────────────────────────────────────────────
# DATA PARAMETERS (fixed for this search)
# ─────────────────────────────────────────────────────────────────────────────────
DATA_START = "2017-11-01"  # Bitcoin price history start

# ─────────────────────────────────────────────────────────────────────────────────
# LABELING PARAMETERS GRID
# ─────────────────────────────────────────────────────────────────────────────────
# horizon: How many days ahead we're predicting
# gauss_window: Size of Gaussian smoothing window (must be odd)
# gauss_std: Standard deviation of Gaussian
LABEL_GRID = {
    "horizon": [1, 3, 5],            # 1, 3, or 5 days ahead
    "gauss_window": [101, 201, 301], # Smoothing window sizes
    "gauss_std": [25.0, 50.0, 100.0], # Gaussian std
}

# ─────────────────────────────────────────────────────────────────────────────────
# FEATURE PARAMETERS GRID
# ─────────────────────────────────────────────────────────────────────────────────
# coord_mode: 'geo' (Earth-centered), 'helio' (Sun-centered), or 'both'
# orb_multiplier: Scale for aspect orbs (lower = stricter)
FEATURE_GRID = {
    "coord_mode": ["both"],           # Start with both geo+helio
    "orb_multiplier": [0.5, 1.0, 1.5], # Aspect orb scaling
    "include_phases": [True],          # Always include moon phases
}

# ─────────────────────────────────────────────────────────────────────────────────
# MODEL HYPERPARAMETERS GRID
# ─────────────────────────────────────────────────────────────────────────────────
MODEL_GRID = {
    "max_depth": [3, 4, 5],           # Tree depth
    "learning_rate": [0.01, 0.03, 0.05], # Step size
    "weight_power": [1.5, 2.0, 2.5],  # Minority class amplification
    "n_estimators": [500],            # Fixed number of trees
    "subsample": [0.8],               # Fixed
    "colsample_bytree": [0.8],        # Fixed
}

# ─────────────────────────────────────────────────────────────────────────────────
# SEARCH SETTINGS
# ─────────────────────────────────────────────────────────────────────────────────
# Set to True for quick test run with reduced grid
TEST_MODE = True  # Set to False for full search

if TEST_MODE:
    print("⚠️ TEST MODE: Using reduced grid for quick testing")
    LABEL_GRID = {
        "horizon": [1],
        "gauss_window": [201],
        "gauss_std": [50.0],
    }
    FEATURE_GRID = {
        "coord_mode": ["both"],
        "orb_multiplier": [1.0],
        "include_phases": [True],
    }
    MODEL_GRID = {
        "max_depth": [4],
        "learning_rate": [0.03],
        "weight_power": [1.5, 2.0],
        "n_estimators": [300],
        "subsample": [0.8],
        "colsample_bytree": [0.8],
    }

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# CALCULATE TOTAL COMBINATIONS
# ═══════════════════════════════════════════════════════════════════════════════════

def count_combinations(grid: Dict[str, List]) -> int:
    """Count total combinations in a parameter grid."""
    total = 1
    for values in grid.values():
        total *= len(values)
    return total

label_combos = count_combinations(LABEL_GRID)
feature_combos = count_combinations(FEATURE_GRID)
model_combos = count_combinations(MODEL_GRID)
total_combos = label_combos * feature_combos * model_combos

print(f"\nGrid Search Statistics:")
print(f"  Label combinations:   {label_combos}")
print(f"  Feature combinations: {feature_combos}")
print(f"  Model combinations:   {model_combos}")
print(f"  ─────────────────────────────")
print(f"  TOTAL COMBINATIONS:   {total_combos}")

## 2. Load Market Data

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# LOAD MARKET DATA (WITH CACHING)
# ═══════════════════════════════════════════════════════════════════════════════════

MARKET_CACHE_PARAMS = {
    "subject_id": cfg.active_subject_id,
    "start_date": DATA_START,
}

df_market = load_cache("market", "data", MARKET_CACHE_PARAMS, verbose=True)

if df_market is None:
    print("Loading from database...")
    df_market = load_market_data()
    df_market = df_market[df_market["date"] >= DATA_START].reset_index(drop=True)
    save_cache(df_market, "market", "data", MARKET_CACHE_PARAMS)

print(f"\nMarket data: {len(df_market)} rows")
print(f"Date range: {df_market['date'].min().date()} → {df_market['date'].max().date()}")

## 3. Initialize Ephemeris

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# INITIALIZE ASTRO SETTINGS
# ═══════════════════════════════════════════════════════════════════════════════════

settings = init_ephemeris()
print(f"Bodies: {[b.name for b in settings.bodies]}")

# Check CUDA availability
use_cuda, device = check_cuda_available()
print(f"Device: {device}")

## 4. Grid Search Functions

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# GRID SEARCH HELPER FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════════════

def generate_grid_combinations(grid: Dict[str, List]) -> List[Dict[str, Any]]:
    """
    Generate all combinations of parameter values from a grid.
    
    Args:
        grid: Dictionary of parameter_name -> list of values
    
    Returns:
        List of dictionaries, each containing one combination
    
    Example:
        grid = {"a": [1, 2], "b": ["x", "y"]}
        result = [{"a": 1, "b": "x"}, {"a": 1, "b": "y"}, ...]
    """
    keys = list(grid.keys())
    values = list(grid.values())
    
    combinations = []
    for combo in itertools.product(*values):
        combinations.append(dict(zip(keys, combo)))
    
    return combinations


def get_or_compute_labels(
    df_market: pd.DataFrame,
    label_params: Dict,
) -> pd.DataFrame:
    """
    Get labels from cache or compute them.
    
    This is the first step where caching saves the most time,
    since label computation involves detrending which is slow.
    """
    cache_params = {
        "subject_id": cfg.active_subject_id,
        "start_date": DATA_START,
        "type": "ternary",
        **label_params,
    }
    
    # Try cache
    df_labels = load_cache("labels", "ternary", cache_params, verbose=False)
    
    if df_labels is None:
        # Compute
        df_labels, threshold = create_ternary_labels(
            df_market,
            horizon=label_params["horizon"],
            gauss_window=label_params["gauss_window"],
            gauss_std=label_params["gauss_std"],
            balance_classes=True,
            verbose=False,
        )
        save_cache(df_labels, "labels", "ternary", cache_params, verbose=False)
    
    return df_labels


def get_or_compute_features(
    df_market: pd.DataFrame,
    settings,
    feature_params: Dict,
) -> pd.DataFrame:
    """
    Get features from cache or compute them.
    
    Features are the most expensive to compute (bodies + aspects + phases).
    Caching here saves significant time in grid search.
    """
    date_range = f"{df_market['date'].min().date()}_{df_market['date'].max().date()}"
    
    cache_params = {
        "coord_mode": feature_params["coord_mode"],
        "orb_mult": feature_params["orb_multiplier"],
        "include_phases": feature_params["include_phases"],
        "date_range": date_range,
    }
    
    # Try cache
    df_features = load_cache("features", "ternary", cache_params, verbose=False)
    
    if df_features is None:
        # ─────────────────────────────────────────────────────────────────────────
        # Compute bodies (most time-consuming)
        # ─────────────────────────────────────────────────────────────────────────
        bodies_cache_params = {
            "coord_mode": feature_params["coord_mode"],
            "date_range": date_range,
        }
        
        df_bodies = load_cache("astro", "bodies", bodies_cache_params, verbose=False)
        geo_by_date = load_cache("astro", "bodies_geo_dict", bodies_cache_params, verbose=False)
        
        if df_bodies is None:
            df_bodies, geo_by_date, helio_by_date = calculate_bodies_for_dates_multi(
                df_market["date"],
                settings,
                coord_mode=feature_params["coord_mode"],
                progress=False,
            )
            save_cache(df_bodies, "astro", "bodies", bodies_cache_params, verbose=False)
            save_cache(geo_by_date, "astro", "bodies_geo_dict", bodies_cache_params, verbose=False)
            if helio_by_date:
                save_cache(helio_by_date, "astro", "bodies_helio_dict", bodies_cache_params, verbose=False)
        
        # ─────────────────────────────────────────────────────────────────────────
        # Compute aspects
        # ─────────────────────────────────────────────────────────────────────────
        aspects_cache_params = {
            "orb_mult": feature_params["orb_multiplier"],
            "coord_mode": feature_params["coord_mode"],
            "date_range": date_range,
        }
        
        df_aspects = load_cache("astro", "aspects", aspects_cache_params, verbose=False)
        
        if df_aspects is None and geo_by_date is not None:
            df_aspects = calculate_aspects_for_dates(
                geo_by_date,
                settings,
                orb_mult=feature_params["orb_multiplier"],
                progress=False,
            )
            save_cache(df_aspects, "astro", "aspects", aspects_cache_params, verbose=False)
        
        # ─────────────────────────────────────────────────────────────────────────
        # Compute phases
        # ─────────────────────────────────────────────────────────────────────────
        phases_cache_params = {"date_range": date_range}
        
        df_phases = None
        if feature_params["include_phases"] and geo_by_date is not None:
            df_phases = load_cache("astro", "phases", phases_cache_params, verbose=False)
            
            if df_phases is None:
                df_phases = calculate_phases_for_dates(geo_by_date, progress=False)
                save_cache(df_phases, "astro", "phases", phases_cache_params, verbose=False)
        
        # ─────────────────────────────────────────────────────────────────────────
        # Build feature matrix
        # ─────────────────────────────────────────────────────────────────────────
        df_features = build_full_features(
            df_bodies,
            df_aspects,
            df_phases=df_phases,
            include_pair_aspects=True,
        )
        
        save_cache(df_features, "features", "ternary", cache_params, verbose=False)
    
    return df_features


def run_single_experiment(
    df_dataset: pd.DataFrame,
    model_params: Dict,
    device: str,
) -> Dict:
    """
    Run a single training experiment with given parameters.
    
    Returns metrics on test set.
    """
    # ─────────────────────────────────────────────────────────────────────────────
    # Split data
    # ─────────────────────────────────────────────────────────────────────────────
    train_df, val_df, test_df = split_dataset(df_dataset, train_ratio=0.7, val_ratio=0.15)
    
    feature_cols = get_feature_columns(df_dataset)
    X_train, y_train = prepare_xy(train_df, feature_cols)
    X_val, y_val = prepare_xy(val_df, feature_cols)
    X_test, y_test = prepare_xy(test_df, feature_cols)
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Extract weight_power from model_params
    # ─────────────────────────────────────────────────────────────────────────────
    weight_power = model_params.pop("weight_power", 2.0)
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Train model
    # ─────────────────────────────────────────────────────────────────────────────
    model = train_xgb_model(
        X_train, y_train,
        X_val, y_val,
        feature_cols,
        n_classes=3,
        device=device,
        weight_power=weight_power,
        verbose=False,
        **model_params,
    )
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Evaluate on test set
    # ─────────────────────────────────────────────────────────────────────────────
    X_test_scaled = model.scaler.transform(X_test)
    y_proba = model.model.predict_proba(X_test_scaled)
    y_pred = y_proba.argmax(axis=1)
    
    metrics = calc_metrics(y_test, y_pred, labels=[0, 1, 2])
    
    # Add weight_power back for logging
    model_params["weight_power"] = weight_power
    
    return metrics

## 5. Run Grid Search

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# RUN GRID SEARCH
# ═══════════════════════════════════════════════════════════════════════════════════

# Generate all parameter combinations
label_combos_list = generate_grid_combinations(LABEL_GRID)
feature_combos_list = generate_grid_combinations(FEATURE_GRID)
model_combos_list = generate_grid_combinations(MODEL_GRID)

print(f"\nStarting grid search with {total_combos} combinations...")
print(f"Device: {device}")
start_time = time.time()

# Results storage
results = []

# Progress bar
pbar = tqdm(total=total_combos, desc="Grid Search")

for label_params in label_combos_list:
    # ─────────────────────────────────────────────────────────────────────────────
    # Get or compute labels for this configuration
    # ─────────────────────────────────────────────────────────────────────────────
    df_labels = get_or_compute_labels(df_market, label_params)
    
    for feature_params in feature_combos_list:
        # ─────────────────────────────────────────────────────────────────────────
        # Get or compute features for this configuration
        # ─────────────────────────────────────────────────────────────────────────
        df_features = get_or_compute_features(df_market, settings, feature_params)
        
        # ─────────────────────────────────────────────────────────────────────────
        # Merge features with labels
        # ─────────────────────────────────────────────────────────────────────────
        df_dataset = merge_features_with_labels(df_features, df_labels)
        
        for model_params in model_combos_list:
            try:
                # ─────────────────────────────────────────────────────────────────
                # Run experiment
                # ─────────────────────────────────────────────────────────────────
                model_params_copy = model_params.copy()  # Don't modify original
                metrics = run_single_experiment(df_dataset, model_params_copy, device)
                
                # ─────────────────────────────────────────────────────────────────
                # Store results
                # ─────────────────────────────────────────────────────────────────
                result = {
                    **label_params,
                    **feature_params,
                    **model_params,
                    "bal_acc": metrics["bal_acc"],
                    "f1_macro": metrics["f1_macro"],
                    "mcc": metrics["mcc"],
                    "recall_down": metrics["recall_down"],
                    "recall_up": metrics["recall_up"],
                    "recall_min": metrics["recall_min"],
                    "recall_gap": metrics["recall_gap"],
                }
                results.append(result)
                
                # Update progress bar with best score so far
                pbar.set_postfix({
                    "best_bal_acc": max(r["bal_acc"] for r in results),
                })
                
            except Exception as e:
                print(f"\n⚠️ Error: {e}")
                pbar.set_postfix({"error": str(e)[:20]})
            
            pbar.update(1)

pbar.close()

elapsed = time.time() - start_time
print(f"\n✓ Grid search completed in {elapsed/60:.1f} minutes")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# ANALYZE RESULTS
# ═══════════════════════════════════════════════════════════════════════════════════

df_results = pd.DataFrame(results)

if not df_results.empty:
    # Sort by balanced accuracy
    df_results = df_results.sort_values("bal_acc", ascending=False)
    
    print("\n" + "=" * 80)
    print("TOP 10 CONFIGURATIONS BY BALANCED ACCURACY")
    print("=" * 80)
    
    display_cols = [
        "horizon", "gauss_window", "orb_multiplier", "weight_power",
        "max_depth", "learning_rate",
        "bal_acc", "f1_macro", "recall_min", "recall_gap"
    ]
    print(df_results[display_cols].head(10).to_string(index=False))
    
    print("\n" + "=" * 80)
    print("TOP 10 CONFIGURATIONS BY RECALL_MIN (balanced across classes)")
    print("=" * 80)
    
    df_by_recall = df_results.sort_values(
        ["recall_min", "recall_gap"], 
        ascending=[False, True]
    )
    print(df_by_recall[display_cols].head(10).to_string(index=False))
else:
    print("⚠️ No results to analyze")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# SAVE RESULTS TO CSV
# ═══════════════════════════════════════════════════════════════════════════════════

if not df_results.empty:
    reports_dir = PROJECT_ROOT / "RESEARCH" / "reports"
    reports_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_path = reports_dir / f"grid_search_ternary_{timestamp}.csv"
    
    df_results.to_csv(report_path, index=False)
    print(f"\n✓ Results saved to: {report_path}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# BEST PARAMETERS SUMMARY
# ═══════════════════════════════════════════════════════════════════════════════════

if not df_results.empty:
    best = df_results.iloc[0]
    
    print("\n" + "=" * 80)
    print("BEST CONFIGURATION")
    print("=" * 80)
    
    print("\nLabeling Parameters:")
    print(f"  horizon:      {best['horizon']}")
    print(f"  gauss_window: {best['gauss_window']}")
    print(f"  gauss_std:    {best.get('gauss_std', 'N/A')}")
    
    print("\nFeature Parameters:")
    print(f"  coord_mode:     {best['coord_mode']}")
    print(f"  orb_multiplier: {best['orb_multiplier']}")
    
    print("\nModel Parameters:")
    print(f"  max_depth:      {best['max_depth']}")
    print(f"  learning_rate:  {best['learning_rate']}")
    print(f"  weight_power:   {best['weight_power']}")
    
    print("\nMetrics:")
    print(f"  Balanced Accuracy: {best['bal_acc']:.4f}")
    print(f"  F1 Macro:          {best['f1_macro']:.4f}")
    print(f"  MCC:               {best['mcc']:.4f}")
    print(f"  Recall DOWN:       {best['recall_down']:.4f}")
    print(f"  Recall UP:         {best['recall_up']:.4f}")
    print(f"  Recall MIN:        {best['recall_min']:.4f}")
    print(f"  Recall GAP:        {best['recall_gap']:.4f}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# SHOW CACHED FILES
# ═══════════════════════════════════════════════════════════════════════════════════

print("\n📁 Cached files after grid search:")
cache_df = list_cache()
if not cache_df.empty:
    print(cache_df[["category", "name", "params", "size_mb"]].to_string(index=False))
    print(f"\nTotal cache size: {cache_df['size_mb'].sum():.2f} MB")
else:
    print("  No cached files found")

## Summary

Grid search complete! The results show the best parameter combinations for:

- **Labeling**: horizon, gauss_window (trend smoothing)
- **Features**: coord_mode (geo/helio), orb_multiplier (aspect strictness)
- **Model**: max_depth, learning_rate, weight_power (class balance)

Use the best parameters in `main_pipeline_ternary.py` for production model.